In [23]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

In [24]:
%run ../utils/helpers.ipynb

In [25]:
# =============================================================================
# SECTION 1: LOAD RAW EVENT LOG
# =============================================================================

def load_event_log(filepath: str) -> pd.DataFrame:
    """
    Load the raw event log CSV file.
    
    WHY: The event log is the foundation of process mining.
    Each row = one event (milestone) in a patient's ED journey.
    
    Parameters:
    -----------
    filepath : str
        Path to the CSV file
        
    Returns:
    --------
    pd.DataFrame : Raw event log
    """
    print("📥 Loading event log...")
    
    df = pd.read_csv(filepath)
    
    print(f"   ✓ Loaded {len(df):,} events")
    print(f"   ✓ {df['visit_id'].nunique():,} unique patient visits")
    print(f"   ✓ Columns: {list(df.columns)}")
    
    return df

In [26]:
# =============================================================================
# SECTION 2: PARSE TIMESTAMPS
# =============================================================================

def parse_timestamps(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert timestamp strings to datetime objects.
    
    WHY: Accurate timestamps are CRITICAL for:
    - Calculating wait times (PIA = Registration → Assessment)
    - Identifying bottlenecks (where patients wait longest)
    - Detecting anomalies (unusually long waits)
    
    The timestamp format is: DD/MM/YY H:MM (e.g., "12/05/21 1:16")
    """
    print("\n⏰ Parsing timestamps...")
    
    df = df.copy()
    
    # Convert to datetime - format is day/month/year hour:minute
    df['timestamp'] = pd.to_datetime(
        df['timestamp'], 
        format='%d/%m/%y %H:%M',
        errors='coerce'  # Handle any malformed timestamps gracefully
    )
    
    # Check for parsing failures
    null_timestamps = df['timestamp'].isna().sum()
    if null_timestamps > 0:
        print(f"   ⚠ Warning: {null_timestamps} timestamps could not be parsed")
    else:
        print(f"   ✓ All timestamps parsed successfully")
    
    # Extract useful time components for analysis
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.day_name()
    df['date'] = df['timestamp'].dt.date
    
    print(f"   ✓ Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    
    return df


In [27]:
# =============================================================================
# SECTION 3: HANDLE MISSING DATA
# =============================================================================

def handle_missing_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Handle missing values in key columns.
    
    WHY: Missing data can skew analysis:
    - Missing zones → can't identify bottleneck locations
    - Missing triage codes → can't prioritize by acuity
    
    Strategy:
    - initial_zone: Fill with 'Unknown' (tracked separately)
    - consult_desc: Missing is meaningful (no consult needed)
    - triage_code: Critical - flag but don't impute
    """
    print("\n🔧 Handling missing data...")
    
    df = df.copy()
    
    # Check missingness in key columns
    missing_report = {
        'initial_zone': df['initial_zone'].isna().sum(),
        'triage_code': df['triage_code'].isna().sum(),
        'consult_desc': df['consult_desc'].isna().sum(),
        'age': df['age'].isna().sum()
    }
    
    for col, count in missing_report.items():
        pct = (count / len(df)) * 100
        print(f"   • {col}: {count:,} missing ({pct:.1f}%)")
    
    # Fill missing zones with 'Unknown' - we'll track these separately
    df['initial_zone'] = df['initial_zone'].fillna('Unknown')
    
    # consult_desc: Missing means no consult - this is informative, not an error
    df['consult_desc'] = df['consult_desc'].fillna('No Consult')
    
    # Create flag for visits with missing zone (for quality tracking)
    df['zone_missing_flag'] = df['initial_zone'] == 'Unknown'
    
    print(f"   ✓ Missing zones filled with 'Unknown'")
    print(f"   ✓ Missing consults marked as 'No Consult'")
    
    return df


In [28]:
# =============================================================================
# SECTION 4: STANDARDIZE COLUMN MAPPINGS
# =============================================================================

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Ensure consistent column naming and create process mining columns.
    
    WHY: Process mining tools expect specific columns:
    - case_id: Unique patient visit identifier
    - activity: The event/milestone name
    - timestamp: When the event occurred
    - resource: Where it happened (zone)
    
    This makes our data compatible with pm4py and other tools.
    """
    print("\n📋 Standardizing columns for process mining...")
    
    df = df.copy()
    
    # Create process mining standard columns (keep originals too)
    df['case_id'] = df['visit_id']          # Process mining case identifier
    df['activity'] = df['event']             # Activity/event name
    df['resource'] = df['initial_zone']      # Resource = zone in ED context
    
    # Standardize triage descriptions for cleaner display
    triage_map = {
        1: '1-RESUSCITATION',
        2: '2-EMERGENCY', 
        3: '3-URGENT',
        4: '4-LESS URGENT',
        5: '5-NON-URGENT'
    }
    df['triage_level'] = df['triage_code'].map(triage_map)
    
    # Create admission flag (key outcome for ML prediction)
    # Disposition codes 7, 8, 9 indicate admission based on disposition_desc patterns
    df['is_admitted'] = df['disposition_desc'].str.contains(
        'Admit|admit', 
        case=False, 
        na=False
    ).astype(int)
    
    # Create LWBS flag (patients who left without being seen/treated)
    df['is_lwbs'] = df['disposition_desc'].str.contains(
        'Left', 
        case=False, 
        na=False
    ).astype(int)
    
    print(f"   ✓ Created process mining columns: case_id, activity, resource")
    print(f"   ✓ Created outcome flags: is_admitted, is_lwbs")
    
    return df

In [29]:
# =============================================================================
# SECTION 5: SORT EVENTS BY TIME (Handle Simultaneous Events)
# =============================================================================

def sort_events(df: pd.DataFrame) -> pd.DataFrame:
    """
    Sort events by case_id and timestamp with logical ordering for ties.
    
    WHY: When events have the same timestamp (e.g., Ambulance Arrival and Triage),
    we need a logical order for process discovery. The clinical order should be:
    
    1. Ambulance Arrival (if present)
    2. Triage
    3. Registration  
    4. Ambulance Transfer (handoff complete)
    5. Assessment (PIA)
    6. Consult Request
    7. Consult Arrival
    8. Discharge
    9. Left ED
    
    This ensures DFG edges make clinical sense.
    """
    print("\n🔀 Sorting events with logical ordering...")
    
    df = df.copy()
    
    # Define logical event order for tie-breaking
    event_order = {
        'Ambulance Arrival': 1,
        'Triage': 2,
        'Registration': 3,
        'Ambulance Transfer': 4,
        'Assessment': 5,
        'Consult Request': 6,
        'Consult Arrival': 7,
        'Discharge': 8,
        'Left ED': 9
    }
    
    df['event_order'] = df['activity'].map(event_order)
    
    # Sort by case_id, then timestamp, then logical event order
    df = df.sort_values(
        by=['case_id', 'timestamp', 'event_order'],
        ascending=[True, True, True]
    ).reset_index(drop=True)
    
    print(f"   ✓ Events sorted by case_id → timestamp → logical order")
    
    return df



In [30]:
def create_visit_summary(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create one row per visit with key timestamps and computed features.
    
    WHY THIS IS CRITICAL FOR TRIAGE LEAD:
    - See patient status at a glance
    - Quickly identify who's been waiting too long
    - Predict outcomes based on early information
    
    Features created:
    - Key timestamps: triage_time, registration_time, assessment_time, etc.
    - Wait times: triage_to_registration, registration_to_assessment (PIA)
    - Total LOS: Length of stay in ED
    - Flags: has_consult, is_ambulance, is_admitted, is_lwbs
    """
    print("\n📊 Creating visit-level summary...")
    
    # Pivot events to get timestamp for each activity per visit
    event_times = df.pivot_table(
        index='case_id',
        columns='activity',
        values='timestamp',
        aggfunc='first'  # Take first occurrence if duplicates
    ).reset_index()
    
    # Get static visit attributes (same for all events in a visit)
    visit_attrs = df.groupby('case_id').agg({
        'patient_id': 'first',
        'initial_zone': 'first',
        'age': 'first',
        'gender': 'first',
        'triage_code': 'first',
        'triage_level': 'first',
        'triage_desc': 'first',
        'disposition_code': 'first',
        'disposition_desc': 'first',
        'consult_desc': 'first',
        'is_admitted': 'first',
        'is_lwbs': 'first',
        'cdu_flag': 'first',
        'zone_missing_flag': 'first'
    }).reset_index()
    
    # Merge timestamps with attributes
    visits = visit_attrs.merge(event_times, on='case_id', how='left')
    
    # Rename timestamp columns for clarity
    timestamp_cols = {
        'Ambulance Arrival': 'ambulance_arrival_time',
        'Triage': 'triage_time',
        'Registration': 'registration_time',
        'Ambulance Transfer': 'ambulance_transfer_time',
        'Assessment': 'assessment_time',
        'Consult Request': 'consult_request_time',
        'Consult Arrival': 'consult_arrival_time',
        'Discharge': 'discharge_time',
        'Left ED': 'left_ed_time'
    }
    visits = visits.rename(columns=timestamp_cols)
    
    # =========================================================================
    # COMPUTE KEY WAIT TIMES (in minutes)
    # =========================================================================
    
    # PIA = Registration to Assessment (Physician Initial Assessment)
    # THIS IS THE KEY METRIC THE COO CARES ABOUT
    visits['pia_minutes'] = (
        visits['assessment_time'] - visits['registration_time']
    ).dt.total_seconds() / 60
    
    # Triage to Registration wait
    visits['triage_to_reg_minutes'] = (
        visits['registration_time'] - visits['triage_time']
    ).dt.total_seconds() / 60
    
    # Consult wait time (for cases with consults)
    visits['consult_wait_minutes'] = (
        visits['consult_arrival_time'] - visits['consult_request_time']
    ).dt.total_seconds() / 60
    
    # Total Length of Stay (LOS)
    # Use Left ED time as the definitive end time
    visits['los_minutes'] = (
        visits['left_ed_time'] - visits['triage_time']
    ).dt.total_seconds() / 60
    
    # =========================================================================
    # CREATE USEFUL FLAGS
    # =========================================================================
    
    # Ambulance arrival flag
    visits['is_ambulance'] = visits['ambulance_arrival_time'].notna().astype(int)
    
    # Has consult flag
    visits['has_consult'] = visits['consult_request_time'].notna().astype(int)
    
    # Extract arrival hour for pattern analysis
    visits['arrival_hour'] = pd.to_datetime(visits['triage_time']).dt.hour
    visits['arrival_day'] = pd.to_datetime(visits['triage_time']).dt.day_name()
    visits['arrival_date'] = pd.to_datetime(visits['triage_time']).dt.date
    
    # =========================================================================
    # DATA QUALITY FLAGS
    # =========================================================================
    
    # Flag cases with missing key events (for conformance checking)
    visits['missing_triage'] = visits['triage_time'].isna().astype(int)
    visits['missing_registration'] = visits['registration_time'].isna().astype(int)
    visits['missing_assessment'] = visits['assessment_time'].isna().astype(int)
    
    # =========================================================================
    # DATA QUALITY: Handle negative/invalid wait times
    # =========================================================================
    # Some records have data entry errors (e.g., discharge before triage)
    # Set invalid times to NaN rather than showing misleading negatives
    
    time_cols = ['pia_minutes', 'triage_to_reg_minutes', 'consult_wait_minutes', 'los_minutes']
    for col in time_cols:
        invalid_count = (visits[col] < 0).sum()
        if invalid_count > 0:
            visits.loc[visits[col] < 0, col] = np.nan
            print(f"   ⚠ {col}: {invalid_count} negative values set to NaN")
    
    print(f"   ✓ Created {len(visits):,} visit summaries")
    print(f"   ✓ Key wait times calculated: PIA, Triage-to-Reg, Consult Wait, LOS")
    print(f"   ✓ Flags created: is_ambulance, has_consult, is_admitted, is_lwbs")
    
    return visits


In [31]:
def load_and_prepare_data(filepath: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Main function to load and prepare all data.
    
    Returns:
    --------
    Tuple of:
    - event_log: Cleaned event-level data (for process mining)
    - visits: Visit-level summary (for ML and dashboards)
    """
    print("=" * 60)
    print("NTH-ED DATA LOADING PIPELINE")
    print("=" * 60)
    
    # Step 1: Load raw data
    df = load_event_log(filepath)
    
    # Step 2: Parse timestamps
    df = parse_timestamps(df)
    
    # Step 3: Handle missing data
    df = handle_missing_data(df)
    
    # Step 4: Standardize columns
    df = standardize_columns(df)
    
    # Step 5: Sort events
    event_log = sort_events(df)
    
    # Step 6: Create visit summary
    visits = create_visit_summary(event_log)
    
    # Final summary
    print("\n" + "=" * 60)
    print("DATA PREPARATION COMPLETE")
    print("=" * 60)
    print(f"\n📈 EVENT LOG SUMMARY:")
    print(f"   • Total events: {len(event_log):,}")
    print(f"   • Unique visits: {event_log['case_id'].nunique():,}")
    print(f"   • Event types: {event_log['activity'].nunique()}")
    print(f"   • Zones: {event_log['resource'].nunique()}")
    
    print(f"\n📊 VISIT SUMMARY:")
    print(f"   • Total visits: {len(visits):,}")
    print(f"   • Admission rate: {visits['is_admitted'].mean()*100:.1f}%")
    print(f"   • LWBS rate: {visits['is_lwbs'].mean()*100:.1f}%")
    print(f"   • Ambulance arrivals: {visits['is_ambulance'].mean()*100:.1f}%")
    print(f"   • Median PIA time: {visits['pia_minutes'].median():.0f} minutes")
    print(f"   • Median LOS: {visits['los_minutes'].median():.0f} minutes")
    
    return event_log, visits

In [32]:
# =============================================================================
# SECTION 8: UTILITY FUNCTIONS FOR OTHER MODULES
# =============================================================================

def get_event_sequence(event_log: pd.DataFrame, case_id: int) -> pd.DataFrame:
    """
    Get the ordered event sequence for a specific visit.
    Useful for debugging and case-level analysis.
    """
    return event_log[event_log['case_id'] == case_id][
        ['activity', 'timestamp', 'resource']
    ].sort_values('timestamp')


def get_zone_summary(visits: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize key metrics by zone - helps Triage Lead see zone performance.
    """
    summary = visits.groupby('initial_zone').agg({
        'case_id': 'count',
        'pia_minutes': 'median',
        'los_minutes': 'median',
        'is_admitted': 'mean',
        'is_lwbs': 'mean'
    }).round(2)
    
    summary.columns = ['Visit Count', 'Median PIA (min)', 'Median LOS (min)', 
                       'Admission Rate', 'LWBS Rate']
    return summary.sort_values('Visit Count', ascending=False)


def get_triage_summary(visits: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize key metrics by triage level - priority analysis for Triage Lead.
    """
    summary = visits.groupby('triage_level').agg({
        'case_id': 'count',
        'pia_minutes': 'median',
        'los_minutes': 'median',
        'is_admitted': 'mean',
        'is_lwbs': 'mean'
    }).round(2)
    
    summary.columns = ['Visit Count', 'Median PIA (min)', 'Median LOS (min)',
                       'Admission Rate', 'LWBS Rate']
    return summary

In [33]:
if __name__ == "__main__":
    # Test with the actual data file
    filepath = "/Users/ishaandawra/Desktop/Machine Learning Notes/Machine Learning Projects/Analytics_Colloquia_Project/data/event_log_ED_MMA_2026.csv"
    
    event_log, visits = load_and_prepare_data(filepath)
    
    print("\n" + "=" * 60)
    print("SAMPLE OUTPUT")
    print("=" * 60)
    
    print("\n🔍 Sample Event Log (first 10 rows):")
    print(event_log[['case_id', 'activity', 'timestamp', 'resource', 'triage_level']].head(10))
    
    print("\n🔍 Sample Visit Summary (first 5 rows):")
    print(visits[['case_id', 'initial_zone', 'triage_level', 'pia_minutes', 
                  'los_minutes', 'is_admitted', 'is_lwbs']].head())
    
    print("\n📊 Zone Summary:")
    print(get_zone_summary(visits))
    
    print("\n📊 Triage Level Summary:")
    print(get_triage_summary(visits))

NTH-ED DATA LOADING PIPELINE
📥 Loading event log...
   ✓ Loaded 90,965 events
   ✓ 16,011 unique patient visits
   ✓ Columns: ['visit_id', 'patient_id', 'initial_zone', 'age', 'month', 'day', 'gender', 'triage_code', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'cdu_flag', 'consult_req_flag', 'consult_arrival_flag', 'event', 'timestamp']

⏰ Parsing timestamps...
   ✓ All timestamps parsed successfully
   ✓ Date range: 2021-03-31 23:59:00 to 2021-06-01 17:16:00

🔧 Handling missing data...
   • initial_zone: 1,950 missing (2.1%)
   • triage_code: 3 missing (0.0%)
   • consult_desc: 69,698 missing (76.6%)
   • age: 0 missing (0.0%)
   ✓ Missing zones filled with 'Unknown'
   ✓ Missing consults marked as 'No Consult'

📋 Standardizing columns for process mining...
   ✓ Created process mining columns: case_id, activity, resource
   ✓ Created outcome flags: is_admitted, is_lwbs

🔀 Sorting events with logical ordering...
   ✓ Events sorted by case_id → timestamp → log

Zone Insights:

- Resus has fastest PIA (8 min) but highest admission rate (55%)

- YZ (Yellow Zone) has longest PIA (51 min) - potential bottleneck

- Unknown zone has highest LWBS rate (22%) - data quality issue

In [34]:
print(visits.columns.tolist())

['case_id', 'patient_id', 'initial_zone', 'age', 'gender', 'triage_code', 'triage_level', 'triage_desc', 'disposition_code', 'disposition_desc', 'consult_desc', 'is_admitted', 'is_lwbs', 'cdu_flag', 'zone_missing_flag', 'ambulance_arrival_time', 'ambulance_transfer_time', 'assessment_time', 'consult_arrival_time', 'consult_request_time', 'discharge_time', 'left_ed_time', 'registration_time', 'triage_time', 'pia_minutes', 'triage_to_reg_minutes', 'consult_wait_minutes', 'los_minutes', 'is_ambulance', 'has_consult', 'arrival_hour', 'arrival_day', 'arrival_date', 'missing_triage', 'missing_registration', 'missing_assessment']
